In [ ]:
import os
import json
import time
from typing import TypedDict, Annotated, List, Dict, Any, Optional
from datetime import datetime
from dataclasses import dataclass
from enum import Enum

from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from tavily import TavilyClient
from pydantic import BaseModel, Field, validator

In [ ]:

class SearchDepth(Enum):
    BASIC = "basic"
    ADVANCED = "advanced"

class SearchProvider(Enum):
    TAVILY = "tavily"

@dataclass
class SearchConfig:
    """Configuration for search operations"""
    max_results: int = 5
    search_depth: SearchDepth = SearchDepth.ADVANCED
    include_answer: bool = True
    include_raw_content: bool = False
    cache_ttl: int = 3600  # 1 hour
    timeout: int = 30
    max_retries: int = 3
    enable_verification: bool = True  # Enable information verification
    verification_threshold: float = 0.7  # Minimum consistency score
    cross_check_sources: int = 3  # Minimum sources for cross-checking

class SearchRequest(BaseModel):
    """Validated search request"""
    query: str = Field(..., min_length=1, max_length=500)
    user_id: Optional[str] = None
    session_id: Optional[str] = None
    config: Optional[SearchConfig] = None
    
    @validator('query')
    def validate_query(cls, v):
        if not v.strip():
            raise ValueError("Query cannot be empty")
        return v.strip()

class VerificationResult(BaseModel):
    """Information verification results"""
    consistency_score: float = Field(..., ge=0, le=1)
    confidence_level: str  # "high", "medium", "low"
    conflicting_claims: List[str] = []
    supporting_sources: List[str] = []
    verification_notes: List[str] = []
    fact_checks: Dict[str, Any] = {}

class SearchResponse(BaseModel):
    """Structured search response"""
    query: str
    answer: str
    tavily_answer: Optional[str] = None
    source_count: int
    search_results: Dict[str, Any]
    timestamp: datetime
    duration_ms: int
    cached: bool = False
    provider: SearchProvider
    session_id: Optional[str] = None
    verification: Optional[VerificationResult] = None

class AgentState(TypedDict):
    messages: Annotated[List[Any], "The conversation messages"]
    query: Annotated[str, "The user's search query"]
    search_results: Annotated[Dict, "Search results from Tavily"]
    final_answer: Annotated[str, "Final answer to the user"]
    config: Annotated[SearchConfig, "Search configuration"]
    start_time: Annotated[float, "Request start time"]
    user_id: Annotated[Optional[str], "User identifier"]
    session_id: Annotated[Optional[str], "Session identifier"]
    verification_result: Annotated[Optional[VerificationResult], "Information verification results"]


In [ ]:

class InformationVerifier:
    """Verify information consistency across multiple sources"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def verify_information(self, query: str, search_results: Dict, config: SearchConfig) -> VerificationResult:
        """
        Verify information consistency across sources
        
        Args:
            query: Original search query
            search_results: Results from search
            config: Search configuration
            
        Returns:
            VerificationResult with consistency analysis
        """
        
        if not config.enable_verification:
            return VerificationResult(
                consistency_score=0.5,
                confidence_level="unknown",
                verification_notes=["Verification disabled"]
            )
        
        results = search_results.get("results", [])
        if len(results) < config.cross_check_sources:
            return VerificationResult(
                consistency_score=0.3,
                confidence_level="low",
                verification_notes=[f"Insufficient sources for verification (found {len(results)}, need {config.cross_check_sources})"]
            )
        
        try:
            # Extract key facts from each source
            facts_by_source = self._extract_facts_from_sources(query, results)
            
            # Cross-check facts for consistency
            consistency_analysis = self._analyze_consistency(facts_by_source)
            
            # Evaluate source credibility
            credibility_scores = self._evaluate_source_credibility(results)
            
            # Calculate overall consistency score
            overall_score = self._calculate_consistency_score(
                consistency_analysis, 
                credibility_scores
            )
            
            # Determine confidence level
            confidence_level = self._determine_confidence_level(overall_score)
            
            # Identify conflicts
            conflicts = self._identify_conflicts(consistency_analysis)
            
            return VerificationResult(
                consistency_score=overall_score,
                confidence_level=confidence_level,
                conflicting_claims=conflicts,
                supporting_sources=[result.get("url", "") for result in results[:3]],
                verification_notes=self._generate_verification_notes(consistency_analysis, credibility_scores),
                fact_checks=consistency_analysis
            )
            
        except Exception as e:
            print(f"⚠ Verification failed: {e}")
            return VerificationResult(
                consistency_score=0.4,
                confidence_level="low",
                verification_notes=[f"Verification failed: {str(e)}"]
            )
    
    def _extract_facts_from_sources(self, query: str, results: List[Dict]) -> Dict[str, List[str]]:
        """Extract key facts from each source with robust JSON parsing"""
        
        fact_extraction_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a fact extraction expert. Extract key factual claims from the given text related to the query.

INSTRUCTIONS:
1. Extract only verifiable factual statements
2. Ignore opinions, speculations, or subjective claims
3. Focus on facts directly related to the query
4. Return facts as a JSON list of strings
5. Each fact should be concise and specific

Query: {query}

Text: {content}

IMPORTANT: Return ONLY a valid JSON array, nothing else. Example:
["fact1", "fact2", "fact3"]"""),
            ("human", "Extract facts from this content")
        ])
        
        facts_by_source = {}
        
        for i, result in enumerate(results[:5]):  # Limit to top 5 sources
            content = result.get("content", "")[:1000]  # Limit content length
            if not content.strip():
                continue
                
            try:
                response = self.llm.invoke(
                    fact_extraction_prompt.format_messages(
                        query=query,
                        content=content
                    )
                )
                
                # Parse JSON response with robust error handling
                facts_text = response.content.strip()
                
                # Remove markdown code blocks if present
                if facts_text.startswith("```json"):
                    facts_text = facts_text[7:-3].strip()
                elif facts_text.startswith("```"):
                    facts_text = facts_text[3:-3].strip()
                elif facts_text.startswith("`"):
                    facts_text = facts_text[1:-1].strip()
                
                try:
                    facts = json.loads(facts_text)
                    # Ensure it's a list and clean the data
                    if isinstance(facts, list):
                        # Filter out non-string items and empty strings
                        clean_facts = [str(fact).strip() for fact in facts if fact and str(fact).strip()]
                        facts_by_source[f"source_{i}"] = clean_facts
                    else:
                        print(f"⚠ Source {i}: Expected list, got {type(facts)}")
                        facts_by_source[f"source_{i}"] = []
                        
                except json.JSONDecodeError as je:
                    print(f"⚠ Source {i}: JSON parse error - {je}")
                    print(f"Raw response: {facts_text[:100]}...")
                    facts_by_source[f"source_{i}"] = []
                
            except Exception as e:
                print(f"⚠ Source {i}: Fact extraction failed - {e}")
                facts_by_source[f"source_{i}"] = []
        
        # Print extraction summary
        total_facts = sum(len(facts) for facts in facts_by_source.values())
        print(f"✓ Extracted {total_facts} facts from {len(facts_by_source)} sources")
        
        return facts_by_source
    
    def _analyze_consistency(self, facts_by_source: Dict[str, List[str]]) -> Dict[str, Any]:
        """Analyze consistency across extracted facts with robust parsing"""
        
        all_facts = []
        for source, facts in facts_by_source.items():
            all_facts.extend(facts)
        
        if not all_facts:
            return {
                "consistent_facts": [], 
                "inconsistent_facts": [], 
                "unique_facts": [],
                "confidence_notes": ["No facts extracted"]
            }
        
        consistency_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a fact-checking expert. Analyze the given facts for consistency.

INSTRUCTIONS:
1. Group similar or related facts together
2. Identify contradictions or inconsistencies
3. Note facts that are consistently reported across sources
4. Identify unique facts from single sources

Facts to analyze:
{facts}

IMPORTANT: Return ONLY valid JSON in this exact format:
{{
    "consistent_facts": ["facts that appear in multiple sources or are compatible"],
    "inconsistent_facts": ["facts that contradict each other"],
    "unique_facts": ["facts from only one source"],
    "confidence_notes": ["brief explanations for consistency assessment"]
}}"""),
            ("human", "Analyze fact consistency")
        ])
        
        try:
            response = self.llm.invoke(
                consistency_prompt.format_messages(
                    facts=json.dumps(all_facts, indent=2)
                )
            )
            
            analysis_text = response.content.strip()
            
            # Clean JSON response
            if analysis_text.startswith("```json"):
                analysis_text = analysis_text[7:-3].strip()
            elif analysis_text.startswith("```"):
                analysis_text = analysis_text[3:-3].strip()
            
            try:
                analysis = json.loads(analysis_text)
                
                # Validate and ensure all required keys exist
                required_keys = ["consistent_facts", "inconsistent_facts", "unique_facts", "confidence_notes"]
                for key in required_keys:
                    if key not in analysis or not isinstance(analysis[key], list):
                        analysis[key] = []
                
                print(f"✓ Consistency analysis: {len(analysis['consistent_facts'])} consistent, {len(analysis['inconsistent_facts'])} inconsistent")
                return analysis
                
            except json.JSONDecodeError as je:
                print(f"⚠ Consistency analysis JSON parse error: {je}")
                print(f"Raw response: {analysis_text[:200]}...")
                
        except Exception as e:
            print(f"⚠ Consistency analysis failed: {e}")
        
        # Fallback response
        return {
            "consistent_facts": all_facts[:3],  # Assume first few are consistent
            "inconsistent_facts": [],
            "unique_facts": all_facts[3:],
            "confidence_notes": ["Analysis failed - using fallback"]
        }
    
    def _evaluate_source_credibility(self, results: List[Dict]) -> Dict[str, float]:
        """Evaluate credibility of sources"""
        credibility_scores = {}
        
        for i, result in enumerate(results):
            score = 0.5  # Base score
            url = result.get("url", "").lower()
            title = result.get("title", "").lower()
            
            # Domain-based scoring
            if any(domain in url for domain in [".edu", ".gov", ".org"]):
                score += 0.3
            elif any(domain in url for domain in [".com", ".net"]):
                score += 0.1
            
            # Known credible sources
            credible_domains = [
                "wikipedia.org", "britannica.com", "nature.com", 
                "sciencedirect.com", "pubmed.ncbi.nlm.nih.gov",
                "who.int", "cdc.gov", "fda.gov", "nih.gov",
                "reuters.com", "bbc.com", "apnews.com", "mayo.edu",
                "harvard.edu", "stanford.edu", "webmd.com"
            ]
            
            if any(domain in url for domain in credible_domains):
                score += 0.2
            
            # Content quality indicators
            content_length = len(result.get("content", ""))
            if content_length > 500:
                score += 0.1
            
            # Tavily relevance score
            tavily_score = result.get("score", 0)
            score += min(tavily_score * 0.2, 0.2)
            
            # Ensure score is between 0 and 1
            credibility_scores[f"source_{i}"] = min(max(score, 0.0), 1.0)
        
        return credibility_scores
    
    def _calculate_consistency_score(
        self, 
        consistency_analysis: Dict[str, Any], 
        credibility_scores: Dict[str, float]
    ) -> float:
        """Calculate overall consistency score"""
        
        consistent_facts = len(consistency_analysis.get("consistent_facts", []))
        inconsistent_facts = len(consistency_analysis.get("inconsistent_facts", []))
        total_facts = consistent_facts + inconsistent_facts + len(consistency_analysis.get("unique_facts", []))
        
        if total_facts == 0:
            return 0.5
        
        # Base consistency ratio
        if consistent_facts + inconsistent_facts == 0:
            consistency_ratio = 0.5
        else:
            consistency_ratio = consistent_facts / (consistent_facts + inconsistent_facts)
        
        # Weight by source credibility
        avg_credibility = sum(credibility_scores.values()) / len(credibility_scores) if credibility_scores else 0.5
        
        # Penalize for conflicts
        conflict_penalty = min(inconsistent_facts * 0.1, 0.3)
        
        # Final score
        final_score = (consistency_ratio * 0.6 + avg_credibility * 0.4) - conflict_penalty
        
        return min(max(final_score, 0.0), 1.0)
    
    def _determine_confidence_level(self, score: float) -> str:
        """Determine confidence level based on score"""
        if score >= 0.8:
            return "high"
        elif score >= 0.6:
            return "medium"
        else:
            return "low"
    
    def _identify_conflicts(self, consistency_analysis: Dict[str, Any]) -> List[str]:
        """Identify specific conflicting claims"""
        return consistency_analysis.get("inconsistent_facts", [])
    
    def _generate_verification_notes(
        self, 
        consistency_analysis: Dict[str, Any], 
        credibility_scores: Dict[str, float]
    ) -> List[str]:
        """Generate human-readable verification notes"""
        notes = []
        
        consistent_count = len(consistency_analysis.get("consistent_facts", []))
        inconsistent_count = len(consistency_analysis.get("inconsistent_facts", []))
        
        notes.append(f"Found {consistent_count} consistent facts across sources")
        
        if inconsistent_count > 0:
            notes.append(f"Detected {inconsistent_count} potential contradictions")
        
        avg_credibility = sum(credibility_scores.values()) / len(credibility_scores) if credibility_scores else 0
        notes.append(f"Average source credibility: {avg_credibility:.2f}")
        
        if any(score > 0.8 for score in credibility_scores.values()):
            notes.append("High-credibility sources found")
        
        return notes


In [ ]:

class SearchAgent:
    """Production-grade search agent with information verification"""
    
    def __init__(
        self,
        google_api_key: Optional[str] = None,
        tavily_api_key: Optional[str] = None
    ):
        # Validate API keys
        self.google_api_key = google_api_key or os.getenv("GOOGLE_API_KEY")
        self.tavily_api_key = tavily_api_key or os.getenv("TAVILY_API_KEY")
        
        if not self.google_api_key:
            raise ValueError("GOOGLE_API_KEY environment variable is required")
        if not self.tavily_api_key:
            raise ValueError("TAVILY_API_KEY environment variable is required")
        
        # Initialize components
        try:
            print("🚀 Initializing SearchAgent...")
            
            self.llm = ChatGoogleGenerativeAI(
                model="gemini-1.5-pro",
                temperature=0.1,
                google_api_key=self.google_api_key
            )
            
            self.tavily_client = TavilyClient(api_key=self.tavily_api_key)
            self.verifier = InformationVerifier(self.llm)
            
            print("✓ API keys validated")
            print("✓ LLM and search client initialized")
            
        except Exception as e:
            raise ValueError(f"Failed to initialize APIs: {str(e)}")
        
        # Create the graph
        self.graph = self._create_graph()
        print("✓ SearchAgent ready!")
    
    def _search_node(self, state: AgentState) -> AgentState:
        """Execute Tavily search"""
        query = state["query"]
        config = state["config"]
        
        try:
            print(f"🔍 Searching: {query}")
            
            search_results = self.tavily_client.search(
                query=query,
                search_depth=config.search_depth.value,
                max_results=config.max_results,
                include_answer=config.include_answer,
                include_raw_content=config.include_raw_content
            )
            
            state["search_results"] = search_results
            source_count = len(search_results.get("results", []))
            print(f"✓ Search completed: {source_count} sources found")
            
        except Exception as e:
            print(f"✗ Search failed: {e}")
            state["search_results"] = {
                "error": str(e),
                "results": [],
                "answer": ""
            }
        
        return state
    
    def _verify_node(self, state: AgentState) -> AgentState:
        """Verify information consistency across sources"""
        
        query = state["query"]
        search_results = state["search_results"]
        config = state["config"]
        
        if not config.enable_verification:
            print("⏭ Verification disabled")
            state["verification_result"] = VerificationResult(
                consistency_score=0.5,
                confidence_level="unknown",
                verification_notes=["Verification disabled"]
            )
            return state
        
        print("🔍 Verifying information consistency...")
        
        try:
            verification_result = self.verifier.verify_information(
                query, search_results, config
            )
            
            state["verification_result"] = verification_result
            
            print(f"✓ Verification completed: {verification_result.confidence_level} confidence ({verification_result.consistency_score:.2f})")
            
            if verification_result.conflicting_claims:
                print(f"⚠ {len(verification_result.conflicting_claims)} conflicts detected")
                for i, conflict in enumerate(verification_result.conflicting_claims[:2], 1):
                    print(f"  {i}. {conflict}")
            else:
                print("✓ No major conflicts detected")
            
        except Exception as e:
            print(f"✗ Verification failed: {e}")
            state["verification_result"] = VerificationResult(
                consistency_score=0.5,
                confidence_level="unknown",
                verification_notes=[f"Verification failed: {e}"]
            )
        
        return state
    
    def _answer_node(self, state: AgentState) -> AgentState:
        """Generate answer with verification context"""
        
        query = state["query"]
        search_results = state["search_results"]
        verification_result = state.get("verification_result")
        
        print("🤖 Generating answer with verification context...")
        
        # Format search results
        if "error" in search_results:
            results_text = f"Search Error: {search_results['error']}"
        else:
            results_text = self._format_search_results(search_results)
        
        # Add verification context
        verification_context = ""
        if verification_result:
            verification_context = f"""
VERIFICATION ANALYSIS:
- Consistency Score: {verification_result.consistency_score:.2f}
- Confidence Level: {verification_result.confidence_level}
- Supporting Sources: {len(verification_result.supporting_sources)}
- Conflicting Claims: {len(verification_result.conflicting_claims)}

{f"CONFLICTS DETECTED: {verification_result.conflicting_claims}" if verification_result.conflicting_claims else ""}

VERIFICATION NOTES:
{chr(10).join(verification_result.verification_notes)}
"""
        
        # Create enhanced prompt with verification
        answer_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a professional AI research assistant with fact-checking capabilities. Provide accurate, well-structured answers based on search results and verification analysis.

CRITICAL GUIDELINES:
1. Use search results as primary information source
2. ALWAYS consider the verification analysis when forming your answer
3. If consistency score is LOW (< 0.6), mention uncertainty and conflicting information
4. If conflicts are detected, acknowledge them explicitly
5. Cite sources with URLs when possible
6. Structure answers with clear sections
7. Provide confidence indicators based on verification

SEARCH RESULTS:
{search_results}

{verification_context}

RESPONSE FORMAT:
- Start with a direct answer to the question
- Include confidence level based on verification
- Mention any limitations or conflicts
- Provide supporting details with source citations
- End with additional context if relevant"""),
            ("human", "Question: {query}")
        ])
        
        try:
            response = self.llm.invoke(
                answer_prompt.format_messages(
                    query=query,
                    search_results=results_text,
                    verification_context=verification_context
                )
            )
            
            state["final_answer"] = response.content
            state["messages"].append(AIMessage(content=response.content))
            
            print("✓ Answer generated with verification context")
            
        except Exception as e:
            error_msg = f"Answer generation failed: {str(e)}"
            state["final_answer"] = error_msg
            state["messages"].append(AIMessage(content=error_msg))
            print(f"✗ Answer generation failed: {e}")
        
        return state
    
    def _format_search_results(self, search_results: Dict) -> str:
        """Format search results for LLM consumption"""
        results_text = ""
        
        # Add Tavily AI answer
        if search_results.get("answer"):
            results_text += f"AI SUMMARY: {search_results['answer']}\n\n"
        
        # Add search results
        if search_results.get("results"):
            results_text += "SOURCES:\n"
            for i, result in enumerate(search_results["results"], 1):
                results_text += f"{i}. {result.get('title', 'Untitled')}\n"
                results_text += f"   URL: {result.get('url', 'No URL')}\n"
                results_text += f"   Content: {result.get('content', 'No content')[:400]}...\n"
                results_text += f"   Relevance Score: {result.get('score', 'N/A')}\n\n"
        
        return results_text
    
    def _create_graph(self) -> StateGraph:
        """Create production workflow with verification"""
        workflow = StateGraph(AgentState)
        
        workflow.add_node("search", self._search_node)
        workflow.add_node("verify", self._verify_node)
        workflow.add_node("answer", self._answer_node)
        
        workflow.set_entry_point("search")
        workflow.add_edge("search", "verify")
        workflow.add_edge("verify", "answer")
        workflow.add_edge("answer", END)
        
        return workflow.compile()
    
    def search(
        self,
        request: SearchRequest,
        config: Optional[SearchConfig] = None
    ) -> SearchResponse:
        """Main search method with full observability"""
        
        start_time = time.time()
        
        # Use provided config or default
        search_config = config or request.config or SearchConfig()
        
        print(f"📊 Verification: {'enabled' if search_config.enable_verification else 'disabled'}")
        
        try:
            # Prepare initial state
            initial_state = {
                "messages": [HumanMessage(content=request.query)],
                "query": request.query,
                "search_results": {},
                "final_answer": "",
                "config": search_config,
                "start_time": start_time,
                "user_id": request.user_id,
                "session_id": request.session_id,
                "verification_result": None
            }
            
            # Execute workflow
            final_state = self.graph.invoke(initial_state)
            
            # Calculate duration
            duration_ms = int((time.time() - start_time) * 1000)
            
            # Create response
            response = SearchResponse(
                query=request.query,
                answer=final_state.get("final_answer", ""),
                tavily_answer=final_state.get("search_results", {}).get("answer"),
                source_count=len(final_state.get("search_results", {}).get("results", [])),
                search_results=final_state.get("search_results", {}),
                timestamp=datetime.now(),
                duration_ms=duration_ms,
                cached=final_state.get("search_results", {}).get("cached", False),
                provider=SearchProvider.TAVILY,
                session_id=request.session_id,
                verification=final_state.get("verification_result")
            )
            
            print(f"⏱ Completed in {duration_ms}ms")
            return response
            
        except Exception as e:
            print(f"❌ Search request failed: {e}")
            raise

# Convenience function for simple usage
def simple_search(query: str, enable_verification: bool = True) -> SearchResponse:
    """Simple search function for quick usage"""
    agent = SearchAgent()
    request = SearchRequest(query=query)
    config = SearchConfig(enable_verification=enable_verification)
    return agent.search(request, config)

In [ ]:

# Example usage with comprehensive testing
if __name__ == "__main__":
    try:
        # Initialize agent
        agent = SearchAgent()
        
        # Test searches with different configurations
        test_cases = [
            {
                "query": "What are the health benefits of intermittent fasting?",
                "config": SearchConfig(enable_verification=True, max_results=5)
            },
            {
                "query": "Latest research on COVID-19 vaccine effectiveness",
                "config": SearchConfig(enable_verification=True, max_results=7, cross_check_sources=4)
            },
            {
                "query": "Dr. Anthony Fauci research background and publications",
                "config": SearchConfig(enable_verification=True, search_depth=SearchDepth.ADVANCED)
            }
        ]
        
        for i, test_case in enumerate(test_cases, 1):
            print(f"\n{'='*80}")
            print(f"🧪 TEST {i}/{len(test_cases)}")
            print('='*80)
            
            try:
                # Create search request
                request = SearchRequest(
                    query=test_case["query"],
                    user_id=f"test_user_{i}",
                    session_id=f"session_{i}"
                )
                
                # Execute search
                result = agent.search(request, test_case["config"])
                
                print(f"\n📋 RESULTS SUMMARY:")
                print(f"Query: {result.query}")
                print(f"Sources: {result.source_count}")
                print(f"Duration: {result.duration_ms}ms")
                print(f"Provider: {result.provider.value}")
                
                if result.verification:
                    print(f"Verification: {result.verification.confidence_level} confidence ({result.verification.consistency_score:.2f})")
                    if result.verification.conflicting_claims:
                        print(f"⚠ Conflicts: {len(result.verification.conflicting_claims)} detected")
                        for j, conflict in enumerate(result.verification.conflicting_claims[:2], 1):
                            print(f"  {j}. {conflict}")
                    else:
                        print("✓ No conflicts detected")
                
                print(f"\n💬 ANSWER PREVIEW:")
                answer_preview = result.answer[:300] + "..." if len(result.answer) > 300 else result.answer
                print(answer_preview)
                
            except Exception as e:
                print(f"❌ Test {i} failed: {e}")
                continue
        
        print(f"\n🎉 All tests completed!")
        
        # Example of simple usage
        print(f"\n{'='*80}")
        print("📝 SIMPLE USAGE EXAMPLE")
        print('='*80)
        simple_result = simple_search("What are symptoms of diabetes?")
        print(f"Simple search result: {simple_result.answer[:200]}...")
        
    except Exception as e:
        print(f"❌ Failed to initialize SearchAgent: {e}")
        print("\n💡 Make sure you have set these environment variables:")
        print("   export GOOGLE_API_KEY='your_google_api_key'")
        print("   export TAVILY_API_KEY='your_tavily_api_key'")
        print("\n📦 Required dependencies:")
        print("   pip install langgraph langchain-google-genai tavily-python pydantic")